In [ ]:
################################################################################################
# sentence-bert와 Faiss 라이브러리를 이용하여 검색 MRR(Mean Reciprocal Rank) 측정하는 예시임
# => MRR 측정 말뭉치는 KorQuAD_v1.0_dev.json 말뭉치를 이용함.
#
# 1. 검색모델 로딩
# 2. KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
# 3. 정답리스트에 대해 임베딩 벡터 생성 후 FAISS에 인덱싱함.
# 4. 쿼리를 list로 만들고, 검색 후 예측된 결과를 list로 만듬
# 5. 정답리스트와 예측리스트를 가지고 MRR을 구함
################################################################################################
import faiss
import numpy as np
import pandas as pd
import time

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything

device = GPU_info()
seed_everything(111)

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
query_num = 0            # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
faiss_search_k = 10         # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
use_cross_encoder = True   # cross_encoder 사용할지 유.무 (true=사용함, false=사용안함)
#-------------------------------------------------------------------------------------
# 1.sentence bert 모델 로딩
#-------------------------------------------------------------------------------------
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from sentence_transformers.cross_encoder import CrossEncoder

bi_encoder_path = "bongsoo/kpf-sbert-v1.1"

# 임베딩 벡터 폴링 모드 선택 (*아래값중 문자열로 입력함, 기본=mean)
# mean=단어 평균, max=최대값, cls=문장, 
#['mean', 'max', 'cls', 'weightedmean', 'lasttoken']
pooling_mode = 'mean'

word_embedding_model = models.Transformer(bi_encoder_path, max_seq_length=256, do_lower_case=True, tokenizer_name_or_path=bi_encoder_path)
#pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())  
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode=pooling_mode)  
bi_encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model])
#bi_encoder = SentenceTransformer(bi_encoder_path)
bi_encoder.to(device)

# cross-encoder 모델 로딩
if use_cross_encoder == True:
    cross_encoder_path = "bongsoo/kpf-cross-encoder-v1"
    cross_encoder = CrossEncoder(cross_encoder_path, max_length=512, device=device)


In [ ]:
#-------------------------------------------------------------------------------------------------------
# 2. KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------

import json
import numpy as np
import pandas as pd

'''
# 사용하지 않음.
def read_json_sample(filepath):
    titles = []
    paragraphs = []
    
    with open(filepath, 'r') as f:
        data = json.load(f)
        for idx, item in enumerate(data):
            paragraphs.append([idx, item['title'], item['paragraph']])

    return paragraphs

jsonfile = '../../elasticsearch/data/KorQuAD_v1.0_train_convert.json'
paragraphs = read_json(jsonfile)
print(paragraphs[0:2])

# paragraphs 리스트릴 => dataframe으로 만듬.
df = pd.DataFrame(paragraphs, columns=['uid','title','paragraph'])
print(df['title'][0:5].values)
'''

#==================================================================================================================
# KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 리스트들을 출력하는 함수
# => IN : KorQuAD_v1.0_dev.json 경로
# => OUT : contexts(문장 리스트), questions(질의리스트), answers(답변리스트), 
#          contextid(문장 id 리스트 : 10001 부터시작), pcontextid(질의와 연관된 문장 id 리스트: 10001부터 시작)
#==================================================================================================================
def read_json(filepath):
    
    context_list = []
    question_list = []
    qcontextid_list = []
    answer_list = []
    contextid_list = []
    
    # KorQuAD_v1.0_train.json 파일을 불러옴
    json_data = json.load(open(filepath, "r", encoding="utf-8"))["data"]

    # KorQuAD_v1.0 포멧에 맞게 파싱하여, context, question, answer 목록들을 구함.
    context_id = 10000
    for entry in json_data:
            for paragraph in entry["paragraphs"]:
                context_text = paragraph["context"]
                context_id += 1
                context_list.append(context_text)
                contextid_list.append(context_id)
                
                for qa in paragraph["qas"]:
                    question_text = qa["question"]
                    
                    for answer in qa["answers"]:
                        answer_text = answer["text"]
                        start_position_character = answer["answer_start"]

                        # question, context, answer, startposition 등을 설정함
                        if question_text and answer_text and context_text and start_position_character:
                            question_list.append(question_text)
                            answer_list.append(answer_text)
                            qcontextid_list.append(context_id)
                            
    return context_list, question_list, answer_list, contextid_list, qcontextid_list
#==================================================================================================================

# korQuad 파일을 불러옴.
jsonfile = './data/KorQuAD_v1.0_dev.json'
contexts, questions, answers, contextids, qcontextids = read_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])


# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [ ]:
df_contexts.head()

In [ ]:
df_questions.head()

In [ ]:
print(df_contexts.shape)
print(df_questions.shape)

In [ ]:
#---------------------------------------------------------
# 3. 임베딩 벡터 생성 후 FAISS에 인덱싱함.
#---------------------------------------------------------

# 리스트 중복 제거 (순서유지 안함)
def remove_duplicates(lst):
    return list(set(lst))

# 리스트 중복 제거 (순서유지함)
def remove_duplicates1(lst):
    return list(dict.fromkeys(lst))

#=============================================================================
# 임베딩 벡터 생성하여 FAISS에 인덱싱하고 ID와 매핑 처리하는 함수
# => IN : contextdf
# => OUT : FASSI index
#=============================================================================
def embeddingforfaiss(df):
           
    # embedding 생성(인코딩)
    start = time.time()
    embeddings = bi_encoder.encode(df.context.to_list(), show_progress_bar=True, convert_to_tensor=False)

    #float32 로 embeddings 타입 변경
    embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

    # instance index 생성
    index = faiss.IndexFlatL2(embeddings.shape[1])

    # id를 매핑 시켜줌 => 이때 idtype은 반드시 int64 type이어야 함.
    index = faiss.IndexIDMap2(index)
    index.add_with_ids(embeddings, df.contextid.values)

    print(f'인코딩 시간 : {time.time()-start:.4f}')
    
    return index

In [ ]:
#df 임베딩 구하고 faiss에 추가함.
index = embeddingforfaiss(df_contexts)


In [ ]:
# 테스트로 여러문장 쿼리 해봄
user_query = ["파우스트_서곡", 
             "카카오_(기업)",
             "공룡의_발견",
             "미국의_정치",
             "인공지능"]

vector = bi_encoder.encode(user_query)
distance, idx = index.search(np.array(vector).astype("float32"), k=3)

#print(distance)
#print(idx)
#print([list(df[df.idx == num]['text']) for num in idx[0]])
#print([list(df[df.idx == num]['text']) for num in idx[1]])

for i, query in enumerate(user_query):
    print(f'Q: {query}')
    count = 0
    for num in idx[i]:
        context = df_contexts[df_contexts.contextid == num]['context'].values
        print(f'{num}, {context[0]}({distance[i][count]:.3f})')
        count += 1
    print('\n')

In [ ]:
#----------------------------------------------------------------
# 4. 쿼리를 list로 만들고, 검색 후 예측된 결과를 list로 만듬
#----------------------------------------------------------------

from tqdm.notebook import tqdm

# Query를 list로 만들고 -> query 인코딩후->검색 결과 출력

if query_num == 0:   # query_num = 0 이면 모든 쿼리(5533개)
    user_query = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 쿼리
    user_query = df_questions['question'][:query_num].values.tolist()
# => K=20으로 함.

vector = bi_encoder.encode(user_query)
distance, idx = index.search(np.array(vector).astype("float32"), k=faiss_search_k)

# 예측검색결과를 리스트로 만듬.
bi_predictions_list = []

for i, query in enumerate(tqdm(user_query)):
    bi_predictions_list.append(idx[i].tolist())
    
print(f'----------------------------------------------------------')
print(f'bi-encoder 예측:{len(bi_predictions_list)}')
print(bi_predictions_list[0:3])

# cross-encoder 사용인 경우에
# - 한번더 검색된 결과에 {쿼리, 문장} 쌍으로 만들어서 cross-encoder로 스코어 출력함.
if use_cross_encoder == True:
    # {쿼리, 문장} 쌍 만듬.
    #count = 0
    cross_predictions_list = []
    for i, predicts in enumerate(tqdm(bi_predictions_list)):
        sentence_combinations = []
        query = user_query[i]
        #count += 1
             
        for predict in predicts:  
             sentence_combinations.append([query, df_contexts[df_contexts.contextid == predict]['context'].values.tolist()[0]])
                     
        # cross-enocoder 돌려서 출력된 score 추가함.
        cross_scores = cross_encoder.predict(sentence_combinations)+1
        #print(f'*cross_scores:{cross_scores}')
        
        dec_cross_idx = reversed(np.argsort(cross_scores))
        tmp_list = []
        for idx in dec_cross_idx:
            #print(f'idx:{idx}-predicts:{predicts[idx]}')
            tmp_list.append(predicts[idx])
         
        cross_predictions_list.append(tmp_list)   
    
    print(f'----------------------------------------------------------')
    print(f'cross-encoder 예측:{len(cross_predictions_list)}')
    print(f'{cross_predictions_list[0:3]}')


In [ ]:
# 검색된 결과를 샘플 출력해 봄
num = 2 #출력해볼 쿼리 번호

# 쿼리
print(f'Q: {df_questions["question"][num]}')

# 정답 출력
qcontextid = df_questions["contextid"][num]
print(f'정답=========================================')
print(f'contextid:{qcontextid}')
print(f'{df_contexts[df_contexts.contextid == qcontextid]["context"].values}')

print(f'예측=============================================')

if use_cross_encoder == True:
    for pcontextid in cross_predictions_list[num]:
        print(f'contextid:{df_contexts[df_contexts.contextid == pcontextid]["contextid"].values}')
        print(f'{df_contexts[df_contexts.contextid == pcontextid]["context"].values}')
else:
    for pcontextid in bi_predictions_list[num]:
        print(f'contextid:{df_contexts[df_contexts.contextid == pcontextid]["contextid"].values}')
        print(f'{df_contexts[df_contexts.contextid == pcontextid]["context"].values}')

In [ ]:
#--------------------------------------------------------------------------------------------------
# 5. MRR을 구함
##--------------------------------------------------------------------------------------------------
#==================================================================================================
# MRR(Mean Reciprocal Rank) 함수
# => IN : ground_truths - 정답 contextid 리스트(예: [10001,10002, 10003, 10004,...])
#         predictions - 예측값 contextid리스트(예: [[10003,10004,...],[10010, 10007,...],[],[],...]
# => OUT : 각 쿼리에 대한 ranks 값 리스트(0~1범위), 전체 평균 쿼리 ranks 값(MRR) 리턴함
#==================================================================================================
def mean_reciprocal_rank(ground_truths, predictions):
    reciprocal_ranks = []
    
    for gt, prediction in zip(ground_truths, predictions):
        rank = 1
        bsearch=False
        for p in prediction:
            #print(f'pred:{p}-gt:{gt}')
            #if p in gt:
            if p == gt:
                reciprocal_ranks.append(1/rank)
                bsearch=True
                #print(f'gt:{gt}=>{1/rank}')
                break
            rank += 1
            
        if bsearch==False:
            reciprocal_ranks.append(0)
            #print(f'gt:{gt}=>0')
       
    # 각 쿼리에 대한 ranks 값(0~1범위), 전체 평균 쿼리 ranks 값(MRR) 리턴함
    return reciprocal_ranks, sum(reciprocal_ranks) / len(reciprocal_ranks)
#==================================================================================================

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
#print(f'gtlen:{len(ground_truths_list)}')
#print(ground_truths_list[0:9])

# MRR를 구함
if use_cross_encoder == True:
    predictions_list = bi_predictions_list
    ranks, score = mean_reciprocal_rank(ground_truths_list, predictions_list)

    # BI-MRR 출력
    print(f'--------------------------------------------------------------------------')
    print('*BI-MRR:{:.4f}'.format(score))
    print(f'*Ranks({len(ranks)}):{ranks[0:20]}')
    
    predictions_list = cross_predictions_list
    ranks, score = mean_reciprocal_rank(ground_truths_list, predictions_list)
    print(f'---------------------------------------------------------------------------')
    print('*CROSS-MRR:{:.4f}'.format(score))
    print(f'*Ranks({len(ranks)}):{ranks[0:20]}')

else:
    predictions_list = bi_predictions_list

    ranks, score = mean_reciprocal_rank(ground_truths_list, predictions_list)

    # BI-MRR 출력
    print(f'----------------------------------------------------------------------------')
    print('*BI-MRR:{:.4f}'.format(score))
    print(f'*Ranks({len(ranks)}):{ranks[0:20]}')